# Exploring the restaurants of Tokyo

A friend of mine, who is currently living in Cancun, Mexico, and running his own Mexican restaurant, contacted me recently. Just a year is left until Tokyo 2020 Olympic Games start, so Tokyo is going to welcome a lot of guests from abroad. My friend was wondering is there any Mexican restaurants in Tokyo, and where are they located? If he decides to open his own Mexian restaurant, which area should he choose?

## Import libraries

In [1]:
import pandas as pd
import numpy as np

#json tools
import json
from pandas.io.json import json_normalize

import requests
from urllib.request import urlopen

#geocoders
from geopy.geocoders import Nominatim

#visualization libraries
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

#kmeans clustering
from sklearn.cluster import KMeans

print('Done!')

Done!


## Create a dataframe 

Reading a CSV file that contains a list of Tokyo's 23 wards into Pandas dataframe:

In [2]:
data = pd.read_csv('23wards.csv')
data.head()

,No.,Name,Population,Latitude,Longitude
0,1,Chiyoda,"59,441",35.694003,139.753594
1,2,Chūō,"147,620",35.666667,139.766667
2,3,Minato,"248,071",35.658069,139.751600
3,4,Shinjuku,"339,211",35.701389,139.709722
4,5,Bunkyō,"223,389",35.716667,139.750000


## Define Foursquare credintials

In [11]:
#set credintials
CLIENT_ID = 'IPTYUZQHVW5OCDTT331BXA1SFQCJ3QCNQ2NVFZHQI5M4ZJLY' # your Foursquare ID
CLIENT_SECRET = '4ARF5SHATZIHJ2FJURJBFIUBZWYKR0UZ4FP5XHAGRE4BCZJ1' # your Foursquare Secret
VERSION = '20190328' # Foursquare API version
print ("Id =",CLIENT_ID)
print("Secret =",CLIENT_SECRET)
print("Version =",VERSION)

Id = IPTYUZQHVW5OCDTT331BXA1SFQCJ3QCNQ2NVFZHQI5M4ZJLY
Secret = 4ARF5SHATZIHJ2FJURJBFIUBZWYKR0UZ4FP5XHAGRE4BCZJ1
Version = 20190328


## Find Mexican restaurants in Tokyo

Define a function that goes through the list of wards and returns all venues that contain the specific search query:

In [12]:
def searchVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            search_query)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Ward Name', 
                  'Ward Latitude', 
                  'Ward Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Search for venues that contain "Mexican":

In [17]:
search_query = 'Mexican'

mexican_venues = searchVenues(names=data['Name'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )

Chiyoda
Chūō
Minato
Shinjuku
Bunkyō
Taitō
Sumida
Kōtō
Shinagawa
Meguro
Ōta
Setagaya
Shibuya
Nakano
Suginami
Toshima
Kita
Arakawa
Itabashi
Nerima
Adachi
Katsushika
Edogawa


In [18]:
#check the dataframe
mexican_venues.head()

,Ward Name,Ward Latitude,Ward Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chiyoda,35.694003,139.753594,TACO LIBRE (タコリブレ),35.694874,139.763678,Mexican Restaurant
1,Chiyoda,35.694003,139.753594,Taco Bell,35.696148,139.759113,Taco Place
2,Chiyoda,35.694003,139.753594,GooooN 手作りトルティーヤ&Bar,35.696316,139.752316,Mexican Restaurant
3,Chiyoda,35.694003,139.753594,ソル・アミーゴ 神保町店,35.696204,139.759998,Mexican Restaurant
4,Chiyoda,35.694003,139.753594,海老とワイン 下町ボラーチョ,35.696174,139.760768,Mexican Restaurant


In [19]:
#how many venues are in the dataframe?
mexican_venues.shape

(327, 7)

Let's find how many Mexican restaurants are there in each ward and order them by descending:

In [20]:
#grouping
mex_count = mexican_venues.groupby('Ward Name').count()[['Venue']]
mex_count.Venue.sum()

327

In [21]:
#sorting
mex_count.sort_values(by='Venue', ascending=False)

,Venue
Ward Name,
Adachi,30
Nakano,30
Taitō,30
Sumida,30
Shinagawa,30
Shibuya,24
Meguro,15
Chiyoda,14
Chūō,14


There are 5 wards with max total count of Mexican venues: Adachi, Nakano, Taito, Sumida, and Shinagawa. 
    Shibuya ward is close to top with its 24 Mexican restaurants.
    
Let's explore and cluster the wards by most common venues to see if there is something common in between those top 5.
Let's pay attention to the wards that have a minimum of Mexican restaurants.

## Clustering the Tokyo wards

A function to get Top 100 venues in each ward:

In [14]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Ward Name', 
                  'Ward Latitude', 
                  'Ward Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Getting the top 100 venues:


In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

tokyo_venues = getNearbyVenues(names=data['Name'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )

Chiyoda
Chūō
Minato
Shinjuku
Bunkyō
Taitō
Sumida
Kōtō
Shinagawa
Meguro
Ōta
Setagaya
Shibuya
Nakano
Suginami
Toshima
Kita
Arakawa
Itabashi
Nerima
Adachi
Katsushika
Edogawa


In [16]:
#checking the size of venues dataframe
tokyo_venues.shape

(2296, 7)

In [23]:
#checking how many unique categories of venues are there
print('There are {} unique categories.'.format(len(tokyo_venues['Venue Category'].unique())))

There are 211 unique categories.


Use OneHot encoding to get unique categories into a dataframe:

In [24]:
# one hot encoding
dt_onehot = pd.get_dummies(tokyo_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['Ward Name'] = tokyo_venues['Ward Name'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

dt_onehot.head()

,Ward Name,African Restaurant,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Used Bookstore,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wagashi Place,Wine Bar,Wine Shop,Yakitori Restaurant,Yoshoku Restaurant,Zoo
0,Chiyoda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Chiyoda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Chiyoda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Chiyoda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Chiyoda,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#checking the dataframe size
dt_onehot.shape

(2296, 212)

Gouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category:

In [26]:
dt_grouped = dt_onehot.groupby('Ward Name').mean().reset_index()
dt_grouped.head()

,Ward Name,African Restaurant,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Used Bookstore,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wagashi Place,Wine Bar,Wine Shop,Yakitori Restaurant,Yoshoku Restaurant,Zoo
0,Adachi,0.0,0.00,0.0,0.01,0.00,0.00,0.0,0.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.01
1,Arakawa,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.00,0.0,0.0,0.00,0.01,0.0,0.0,0.00,0.01,0.00
2,Bunkyō,0.0,0.01,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.00,0.0,0.0,0.01,0.01,0.0,0.0,0.01,0.00,0.00
3,Chiyoda,0.0,0.00,0.0,0.00,0.00,0.02,0.0,0.0,0.0,...,0.02,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.03,0.00
4,Chūō,0.0,0.00,0.0,0.00,0.03,0.00,0.0,0.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.01,0.00


In [27]:
#checking the grouped dataframe size
dt_grouped.shape

(23, 212)

Printing out each neighborhood along with the top 5 most common venues in it:

In [28]:
num_top_venues = 5

for hood in dt_grouped['Ward Name']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Ward Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adachi----
               venue  freq
0  Convenience Store  0.28
1     Discount Store  0.09
2   Ramen Restaurant  0.08
3        Supermarket  0.06
4         Restaurant  0.04


----Arakawa----
                 venue  freq
0    Convenience Store  0.20
1     Ramen Restaurant  0.08
2  Japanese Restaurant  0.06
3             Sake Bar  0.06
4        Grocery Store  0.05


----Bunkyō----
                 venue  freq
0                 Café  0.11
1    Convenience Store  0.09
2     Ramen Restaurant  0.06
3            BBQ Joint  0.06
4  Japanese Restaurant  0.05


----Chiyoda----
                       venue  freq
0                       Café  0.14
1           Ramen Restaurant  0.10
2  Japanese Curry Restaurant  0.06
3              Historic Site  0.05
4                Coffee Shop  0.04


----Chūō----
                 venue  freq
0  Japanese Restaurant  0.12
1     Sushi Restaurant  0.11
2     Ramen Restaurant  0.06
3                 Café  0.05
4          Coffee Shop  0.04


----Edogawa----
     

Converting the results to Pandas dataframe:

In [29]:
#function to sort the venues in descending order:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
#create the new dataframe and display the top 10 venues for each neighborhood:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Ward Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Ward Name'] = dt_grouped['Ward Name']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Ward Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adachi,Convenience Store,Discount Store,Ramen Restaurant,Supermarket,Restaurant,Italian Restaurant,Donburi Restaurant,Sushi Restaurant,Shopping Mall,Chinese Restaurant
1,Arakawa,Convenience Store,Ramen Restaurant,Japanese Restaurant,Sake Bar,Grocery Store,BBQ Joint,Park,Café,Supermarket,Fabric Shop
2,Bunkyō,Café,Convenience Store,Ramen Restaurant,BBQ Joint,Japanese Restaurant,Italian Restaurant,Grocery Store,Japanese Curry Restaurant,Chinese Restaurant,Coffee Shop
3,Chiyoda,Café,Ramen Restaurant,Japanese Curry Restaurant,Historic Site,Bookstore,Coffee Shop,Indian Restaurant,Thai Restaurant,Yoshoku Restaurant,Chinese Restaurant
4,Chūō,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Café,Coffee Shop,Donburi Restaurant,Art Gallery,Sandwich Place,Soba Restaurant,BBQ Joint


Run k-means to cluster the neighborhood into 5 clusters.

In [31]:
# set number of clusters
kclusters = 5

dt_grouped_clustering = dt_grouped.drop('Ward Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 0, 0, 2, 1, 1, 1, 3, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each ward.

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Labels', kmeans.labels_)

dt_merged = data

# merge downtown_grouped with toronto data to add latitude/longitude for each neighborhood
dt_merged = dt_merged.join(neighborhoods_venues_sorted.set_index('Ward Name'), on='Name')

dt_merged.head()

,No.,Name,Population,Latitude,Longitude,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Chiyoda,"59,441",35.694003,139.753594,0,Café,Ramen Restaurant,Japanese Curry Restaurant,Historic Site,Bookstore,Coffee Shop,Indian Restaurant,Thai Restaurant,Yoshoku Restaurant,Chinese Restaurant
1,2,Chūō,"147,620",35.666667,139.766667,2,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Café,Coffee Shop,Donburi Restaurant,Art Gallery,Sandwich Place,Soba Restaurant,BBQ Joint
2,3,Minato,"248,071",35.658069,139.751600,2,Japanese Restaurant,Soba Restaurant,Ramen Restaurant,BBQ Joint,Hotel,Coffee Shop,Chinese Restaurant,Bakery,Park,French Restaurant
3,4,Shinjuku,"339,211",35.701389,139.709722,4,Korean Restaurant,Convenience Store,BBQ Joint,Soba Restaurant,Hotel,Coffee Shop,Grocery Store,Bakery,Park,Japanese Restaurant
4,5,Bunkyō,"223,389",35.716667,139.750000,0,Café,Convenience Store,Ramen Restaurant,BBQ Joint,Japanese Restaurant,Italian Restaurant,Grocery Store,Japanese Curry Restaurant,Chinese Restaurant,Coffee Shop


Visualize the resulting clusters on map:

In [33]:
#get the coordinates for Tokyo
address = 'Tokyo'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Tokyo are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Tokyo are 35.6828387, 139.7594549.


In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Latitude'], dt_merged['Longitude'], dt_merged['Name'], dt_merged['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=9,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Exploring the clusters

In [35]:
#Cluster 1
dt_merged.loc[dt_merged['Labels'] == 0, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Name,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Chiyoda,0,Café,Ramen Restaurant,Japanese Curry Restaurant,Historic Site,Bookstore,Coffee Shop,Indian Restaurant,Thai Restaurant,Yoshoku Restaurant,Chinese Restaurant
4,Bunkyō,0,Café,Convenience Store,Ramen Restaurant,BBQ Joint,Japanese Restaurant,Italian Restaurant,Grocery Store,Japanese Curry Restaurant,Chinese Restaurant,Coffee Shop
9,Meguro,0,Café,Ramen Restaurant,Coffee Shop,Japanese Restaurant,Bakery,Bistro,Italian Restaurant,Dessert Shop,Sake Bar,Indian Restaurant


In [36]:
#Cluster 2
dt_merged.loc[dt_merged['Labels'] == 1, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Name,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Itabashi,1,Convenience Store,Ramen Restaurant,Fast Food Restaurant,Supermarket,Grocery Store,Italian Restaurant,Gym / Fitness Center,Sake Bar,Café,Drugstore
20,Adachi,1,Convenience Store,Discount Store,Ramen Restaurant,Supermarket,Restaurant,Italian Restaurant,Donburi Restaurant,Sushi Restaurant,Shopping Mall,Chinese Restaurant
21,Katsushika,1,Convenience Store,Sake Bar,Discount Store,Park,Sushi Restaurant,Ramen Restaurant,Italian Restaurant,Supermarket,Train Station,Arcade
22,Edogawa,1,Convenience Store,Ramen Restaurant,Restaurant,Italian Restaurant,Fast Food Restaurant,Chinese Restaurant,Supermarket,Sushi Restaurant,Grocery Store,Donburi Restaurant


In [37]:
#Cluster 3
dt_merged.loc[dt_merged['Labels'] == 2, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Name,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chūō,2,Japanese Restaurant,Sushi Restaurant,Ramen Restaurant,Café,Coffee Shop,Donburi Restaurant,Art Gallery,Sandwich Place,Soba Restaurant,BBQ Joint
2,Minato,2,Japanese Restaurant,Soba Restaurant,Ramen Restaurant,BBQ Joint,Hotel,Coffee Shop,Chinese Restaurant,Bakery,Park,French Restaurant
5,Taitō,2,Sake Bar,Ramen Restaurant,Japanese Restaurant,Chinese Restaurant,Café,Japanese Curry Restaurant,Soba Restaurant,BBQ Joint,Science Museum,Bed & Breakfast
6,Sumida,2,Ramen Restaurant,Sake Bar,Coffee Shop,Chinese Restaurant,BBQ Joint,Pub,Thai Restaurant,Hotel,Italian Restaurant,Multiplex
8,Shinagawa,2,Ramen Restaurant,Sake Bar,BBQ Joint,Japanese Restaurant,Park,Coffee Shop,Restaurant,Soba Restaurant,Tonkatsu Restaurant,Convenience Store
12,Shibuya,2,Café,Sushi Restaurant,Ramen Restaurant,Record Shop,Japanese Restaurant,Bar,Italian Restaurant,Yakitori Restaurant,Chinese Restaurant,Sake Bar
13,Nakano,2,Ramen Restaurant,Sake Bar,Café,Japanese Restaurant,Italian Restaurant,BBQ Joint,Japanese Curry Restaurant,Record Shop,Bar,Theater
15,Toshima,2,Sake Bar,Ramen Restaurant,Café,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Chinese Restaurant,Yoshoku Restaurant,Hobby Shop,Steakhouse


In [38]:
#Cluster 4
dt_merged.loc[dt_merged['Labels'] == 3, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Name,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Kōtō,3,Convenience Store,Ramen Restaurant,Japanese Restaurant,Coffee Shop,Sake Bar,Italian Restaurant,BBQ Joint,Park,Sushi Restaurant,Café
10,Ōta,3,Ramen Restaurant,Convenience Store,Chinese Restaurant,Sake Bar,Japanese Restaurant,Tonkatsu Restaurant,Dumpling Restaurant,Sushi Restaurant,Bar,Hot Spring
11,Setagaya,3,Convenience Store,Bakery,Ramen Restaurant,Coffee Shop,Japanese Restaurant,Café,Sushi Restaurant,Italian Restaurant,Soba Restaurant,Tram Station
14,Suginami,3,Convenience Store,Ramen Restaurant,Supermarket,Coffee Shop,Park,Chinese Restaurant,Café,Bakery,Sushi Restaurant,Japanese Restaurant
16,Kita,3,Convenience Store,Ramen Restaurant,Sake Bar,Japanese Restaurant,Donburi Restaurant,Park,Supermarket,Soba Restaurant,Museum,Shopping Mall
17,Arakawa,3,Convenience Store,Ramen Restaurant,Japanese Restaurant,Sake Bar,Grocery Store,BBQ Joint,Park,Café,Supermarket,Fabric Shop
19,Nerima,3,Ramen Restaurant,Convenience Store,Italian Restaurant,Sake Bar,BBQ Joint,Grocery Store,Soba Restaurant,Café,Coffee Shop,Indian Restaurant


In [39]:
#Cluster 5
dt_merged.loc[dt_merged['Labels'] == 4, dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

,Name,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Shinjuku,4,Korean Restaurant,Convenience Store,BBQ Joint,Soba Restaurant,Hotel,Coffee Shop,Grocery Store,Bakery,Park,Japanese Restaurant


### Explore the Mexican restaurants near New Olympic Stadium

Let's find if there are any Mexican restaurants near the New Olympic Stadium.

In [42]:
#set the coordinates for New Olympic Stadium
st_latitude = 35.678056
st_longitude = 139.714722

In [44]:
#set our arguments
search_query = 'Mexican'
radius = 1000

In [45]:
#create the URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, st_latitude, st_longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IPTYUZQHVW5OCDTT331BXA1SFQCJ3QCNQ2NVFZHQI5M4ZJLY&client_secret=4ARF5SHATZIHJ2FJURJBFIUBZWYKR0UZ4FP5XHAGRE4BCZJ1&ll=35.678056,139.714722&v=20190328&query=Mexican&radius=1000&limit=100'

In [46]:
#get results
results = requests.get(url).json()
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Ocho Taqueria Mexican,Mexican Restaurant,神宮前2-3-30,JP,渋谷区,日本,第2竹上ビルB1,418,"[神宮前2-3-30 (第2竹上ビルB1), 渋谷区, 東京都, 150-0001, 日本]","[{'label': 'display', 'lat': 35.674454, 'lng':...",35.674454,139.713400,150-0001,東京都,5c1716b7f1fdaf002cd9d9e4
1,Restaurante Mexicano El Paso,Burrito Place,NaN,JP,NaN,日本,NaN,1125,[日本],"[{'label': 'display', 'lat': 35.66815823267787...",35.668158,139.717265,NaN,NaN,51d26bc6498e525a4ac54150


In [47]:
#get the name of resulting venues
dataframe_filtered.name

0           Ocho Taqueria Mexican
1    Restaurante Mexicano El Paso
Name: name, dtype: object

In [48]:
# generate map centred around the stadium
stadium_map = folium.Map(location=[st_latitude, st_longitude], zoom_start=15) 

# add a red circle marker to represent the stadium
folium.features.CircleMarker(
    [st_latitude, st_longitude],
    radius=15,
    color='red',
    popup='New Olympic Stadium',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(stadium_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(stadium_map)

# display map
stadium_map

### Explore the venues near the stadium

In [49]:
#venue 1 - Ocho Taqueria Mexican
venue_id = '5c1716b7f1fdaf002cd9d9e4'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'pageUpdates', 'inbox', 'attributes'])


{'id': '5c1716b7f1fdaf002cd9d9e4',
 'name': 'Ocho Taqueria Mexican',
 'contact': {},
 'location': {'address': '神宮前2-3-30',
  'crossStreet': '第2竹上ビルB1',
  'lat': 35.674454,
  'lng': 139.7134,
  'labeledLatLngs': [{'label': 'display', 'lat': 35.674454, 'lng': 139.7134}],
  'postalCode': '150-0001',
  'cc': 'JP',
  'city': '渋谷区',
  'state': '東京都',
  'country': '日本',
  'formattedAddress': ['神宮前2-3-30 (第2竹上ビルB1)', '渋谷区, 東京都', '150-0001', '日本']},
 'canonicalUrl': 'https://foursquare.com/v/ocho-taqueria-mexican/5c1716b7f1fdaf002cd9d9e4',
 'categories': [{'id': '4bf58dd8d48988d1c1941735',
   'name': 'Mexican Restaurant',
   'pluralName': 'Mexican Restaurants',
   'shortName': 'Mexican',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0},
 'price': {'tier': 2, 'message': 'Moderate', 'currency': '¥'},
 'likes': {'count': 1,
  'groups': [{'type': 'others',
    'count': 1,
    'ite

In [50]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [51]:
result['response']['venue']['tips']['count']

0

In [52]:
#venue 2 - Restaurante Mexicano El Paso
venue_id = '51d26bc6498e525a4ac54150'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'pageUpdates', 'inbox', 'attributes'])


{'id': '51d26bc6498e525a4ac54150',
 'name': 'Restaurante Mexicano El Paso',
 'contact': {},
 'location': {'lat': 35.66815823267787,
  'lng': 139.7172645620672,
  'labeledLatLngs': [{'label': 'display',
    'lat': 35.66815823267787,
    'lng': 139.7172645620672}],
  'cc': 'JP',
  'country': '日本',
  'formattedAddress': ['日本']},
 'canonicalUrl': 'https://foursquare.com/v/restaurante-mexicano-el-paso/51d26bc6498e525a4ac54150',
 'categories': [{'id': '4bf58dd8d48988d153941735',
   'name': 'Burrito Place',
   'pluralName': 'Burrito Places',
   'shortName': 'Burritos',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/burrito_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0},
 'price': {'tier': 1, 'message': 'Cheap', 'currency': '¥'},
 'likes': {'count': 2,
  'groups': [{'type': 'others',
    'count': 2,
    'items': [{'id': '62614233',
      'firstName': 'LeandroTIM BETA LAB ME ADICIONA QUE TE ACEITO',
      'gender': 'male',
     

In [53]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [54]:
result['response']['venue']['tips']['count']

0

### Which venues are trending near the New Olympic Stadium?

In [55]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, st_latitude, st_longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()

if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)
    
# display trending venues
trending_venues_df

,name,categories,location.distance,location.city,location.postalCode,location.state,location.country,location.lat,location.lng
0,Shinjuku Station (新宿駅),Train Station,1883,東京,NaN,東京都,日本,35.690286,139.700330
1,Shinjuku Expressway Bus Terminal (バスタ新宿),Bus Station,1722,東京,151-0051,東京都,日本,35.688684,139.700877
2,JR Shinjuku Station (JR 新宿駅),Train Station,1896,東京,160-0022,東京都,日本,35.690409,139.700279
3,Shinjuku Piccadilly (新宿ピカデリー),Multiplex,1921,新宿区,160-0022,東京都,日本,35.692814,139.703705
